# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,sosnogorsk,63.6023,53.8817,-9.14,99,96,1.34,RU,1698721554
1,1,susuman,62.7833,148.1667,-13.34,94,82,0.45,RU,1698721554
2,2,adamstown,-25.0660,-130.1015,23.00,76,100,3.06,PN,1698721376
3,3,auki,-8.7676,160.7034,27.86,82,98,0.78,SB,1698721554
4,4,papatowai,-46.5619,169.4708,14.90,66,100,2.51,NZ,1698721554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 27)
& (city_data_df["Max Temp"] > 21) 
& (city_data_df["Cloudiness"] < 3)
& (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
37,37,robertsport,6.7533,-11.3686,26.17,84,1,2.66,LR,1698721560
65,65,longboat key,27.4125,-82.6590,24.45,75,0,2.57,US,1698721556
112,112,maur,30.0833,75.2500,24.53,33,2,2.07,IN,1698721572
133,133,marsa alam,25.0757,34.8918,25.41,67,0,2.16,EG,1698721577
165,165,heathrow,28.7633,-81.3722,23.36,90,0,2.06,US,1698721583
169,169,fort bragg,35.1390,-79.0060,21.91,75,0,2.06,US,1698721359
177,177,mailsi,29.8003,72.1758,26.15,31,0,2.13,PK,1698721585
186,186,salalah,17.0151,54.0924,25.05,83,0,2.06,OM,1698721452
198,198,kuchaiburi,22.2667,86.1667,23.91,67,0,1.22,IN,1698721578
203,203,alamos,27.0167,-108.9333,22.04,47,0,1.14,MX,1698721589


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
37,robertsport,LR,6.7533,-11.3686,84,
65,longboat key,US,27.4125,-82.6590,75,
112,maur,IN,30.0833,75.2500,33,
133,marsa alam,EG,25.0757,34.8918,67,
165,heathrow,US,28.7633,-81.3722,90,
169,fort bragg,US,35.1390,-79.0060,75,
177,mailsi,PK,29.8003,72.1758,31,
186,salalah,OM,17.0151,54.0924,83,
198,kuchaiburi,IN,22.2667,86.1667,67,
203,alamos,MX,27.0167,-108.9333,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key
         }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
   for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")
    
# Display sample data
hotel_df

Starting hotel search
robertsport - nearest hotel: Nana's Lodge
longboat key - nearest hotel: Zota Beach Resort
maur - nearest hotel: No hotel found
marsa alam - nearest hotel: Blue House Hotel
heathrow - nearest hotel: Courtyard by Marriott Orlando Lake Mary/North
fort bragg - nearest hotel: Airborne Inn Lodging
mailsi - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
kuchaiburi - nearest hotel: No hotel found
alamos - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
carnarvon - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
wankyi - nearest hotel: Pony Hotel
saint-pierre - nearest hotel: Tropic Hotel
isla mujeres - nearest hotel: Villa La Bella
aswan - nearest hotel: Yaseen hotel
sebring - nearest hotel: Residence Inn by Marriott Sebring
laveen - nearest hotel: Gila River Resorts & Casinos - Vee Quiva
axim - nearest hotel: Axim Beach Hotel
saint-paul - nearest hotel: Hôtel d'application
gebeit - nearest hote

mailsi - nearest hotel: No hotel found
salalah - nearest hotel: Muscat International Hotel
kuchaiburi - nearest hotel: No hotel found
alamos - nearest hotel: No hotel found
tazacorte - nearest hotel: App Leyma
carnarvon - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
wankyi - nearest hotel: Pony Hotel
saint-pierre - nearest hotel: Tropic Hotel
isla mujeres - nearest hotel: Villa La Bella
aswan - nearest hotel: Yaseen hotel
sebring - nearest hotel: Residence Inn by Marriott Sebring
laveen - nearest hotel: Gila River Resorts & Casinos - Vee Quiva
axim - nearest hotel: Axim Beach Hotel
saint-paul - nearest hotel: Hôtel d'application
gebeit - nearest hotel: No hotel found
leticia - nearest hotel: Waira Suites
surajgarh - nearest hotel: No hotel found
guerrero negro - nearest hotel: Plaza sal paraiso
robertsport - nearest hotel: Nana's Lodge
longboat key - nearest hotel: Zota Beach Resort
maur - nearest hotel: No hotel found
marsa alam - nearest hotel: Blue House H

KeyboardInterrupt: 

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 0.8,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)